In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 🛒 Market Basket Analysis - Adım Adım Tutorial\n",
    "\n",
    "Bu notebook, Market Basket Analysis'in temellerini adım adım öğretmek için hazırlanmıştır.\n",
    "\n",
    "## 📚 Bu Tutorial'da Öğreneceğiniz Konular:\n",
    "1. Market Basket Analysis nedir?\n",
    "2. Support, Confidence, Lift kavramları\n",
    "3. Veri yükleme ve hazırlama\n",
    "4. Popüler ürünleri bulma\n",
    "5. Ürün birlikteliklerini hesaplama\n",
    "6. Association rules oluşturma\n",
    "7. Sonuçları yorumlama\n",
    "8. İş uygulamaları"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. 📖 Market Basket Analysis Nedir?\n",
    "\n",
    "Market Basket Analysis (Market Sepeti Analizi), müşterilerin hangi ürünleri birlikte satın aldığını analiz eden bir veri madenciliği tekniğidir.\n",
    "\n",
    "### 🎯 Amaçları:\n",
    "- **Cross-selling**: Hangi ürünleri birlikte önerebiliriz?\n",
    "- **Ürün yerleştirme**: Mağazada ürünleri nereye koymalıyız?\n",
    "- **Kampanya planlama**: Hangi ürünlerde birlikte indirim yapalım?\n",
    "- **Müşteri davranışı**: Müşteriler nasıl alışveriş yapıyor?\n",
    "\n",
    "### 📊 Temel Kavramlar:\n",
    "- **Support**: Bir ürün/ürün çiftinin ne kadar sık alındığı\n",
    "- **Confidence**: A ürününü alanların kaçı B ürününü de alıyor\n",
    "- **Lift**: İki ürünün birlikte alınma gücü"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Gerekli kütüphaneleri yükleyin\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from itertools import combinations\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')\n",
    "\n",
    "# Grafik ayarları\n",
    "plt.style.use('default')\n",
    "plt.rcParams['figure.figsize'] = (10, 6)\n",
    "plt.rcParams['font.size'] = 12\n",
    "\n",
    "print(\"✅ Kütüphaneler yüklendi!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. 📊 Veri Yükleme ve İnceleme\n",
    "\n",
    "İlk olarak veriyi yükleyelim ve yapısını anlayalım."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Veriyi yükle\n",
    "print(\"📁 Veri yükleniyor...\")\n",
    "veri = pd.read_csv('../data/basket_analysis.csv', index_col=0)\n",
    "\n",
    "print(f\"✅ Veri yüklendi!\")\n",
    "print(f\"📊 Veri boyutu: {veri.shape[0]} sepet, {veri.shape[1]} ürün\")\n",
    "print(f\"🏷️ Ürünler: {list(veri.columns)}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# İlk 5 satırı görelim\n",
    "print(\"🔍 İlk 5 sepet:\")\n",
    "veri.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Veri tipi kontrolü\n",
    "print(\"📋 Veri tipleri:\")\n",
    "print(veri.dtypes.unique())\n",
    "\n",
    "print(\"\\n🔢 Benzersiz değerler:\")\n",
    "print(veri.values.flatten()[:20])"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 💡 Veri Formatı Açıklaması:\n",
    "- Her **satır** bir sepeti (transaction) temsil ediyor\n",
    "- Her **sütun** bir ürünü temsil ediyor  \n",
    "- **True/False** değerleri o ürünün sepette olup olmadığını gösteriyor\n",
    "- Bu format **binary matrix** olarak adlandırılır"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. 🛍️ Sepetleri Hazırlama\n",
    "\n",
    "Binary matrix formatındaki veriyi sepet listelerine çevirelim."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Sepetleri oluştur\n",
    "def sepetleri_olustur(veri):\n",
    "    sepetler = []\n",
    "    \n",
    "    for i, satir in veri.iterrows():\n",
    "        sepet = []\n",
    "        for urun in veri.columns:\n",
    "            if satir[urun] == True or satir[urun] == 'True':\n",
    "                sepet.append(urun)\n",
    "        \n",
    "        if sepet:  # Boş sepetleri ekleme\n",
    "            sepetler.append(sepet)\n",
    "    \n",
    "    return sepetler\n",
    "\n",
    "sepetler = sepetleri_olustur(veri)\n",
    "print(f\"✅ {len(sepetler)} sepet hazırlandı\")\n",
    "\n",
    "# İlk 5 sepeti görelim\n",
    "print(\"\\n🛒 İlk 5 sepet:\")\n",
    "for i, sepet in enumerate(sepetler[:5], 1):\n",
    "    print(f\"{i}. {sepet}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Sepet istatistikleri\n",
    "sepet_boyutlari = [len(sepet) for sepet in sepetler]\n",
    "\n",
    "print(\"📊 SEPET İSTATİSTİKLERİ:\")\n",
    "print(f\"🔸 Toplam sepet sayısı: {len(sepetler)}\")\n",
    "print(f\"🔸 Ortalama ürün/sepet: {np.mean(sepet_boyutlari):.1f}\")\n",
    "print(f\"🔸 En fazla ürün/sepet: {max(sepet_boyutlari)}\")\n",
    "print(f\"🔸 En az ürün/sepet: {min(sepet_boyutlari)}\")\n",
    "print(f\"🔸 Medyan ürün/sepet: {np.median(sepet_boyutlari):.1f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. 📊 Popüler Ürünleri Bulma\n",
    "\n",
    "Hangi ürünler en sık satın alınıyor?"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Her ürünün kaç sepette olduğunu hesapla\n",
    "def urun_popularitesi_hesapla(sepetler):\n",
    "    urun_sayilari = {}\n",
    "    \n",
    "    for sepet in sepetler:\n",
    "        for urun in sepet:\n",
    "            urun_sayilari[urun] = urun_sayilari.get(urun, 0) + 1\n",
    "    \n",
    "    return urun_sayilari\n",
    "\n",
    "urun_sayilari = urun_popularitesi_hesapla(sepetler)\n",
    "\n",
    "# Popülarite sırasına göre sırala\n",
    "sorted_urunler = sorted(urun_sayilari.items(), key=lambda x: x[1], reverse=True)\n",
    "\n",
    "print(\"🏆 EN POPÜLER ÜRÜNLER:\")\n",
    "print(\"Sıra | Ürün        | Sepet Sayısı | Support (%)\")\n",
    "print(\"-\" * 50)\n",
    "\n",
    "for i, (urun, sayi) in enumerate(sorted_urunler, 1):\n",
    "    support = (sayi / len(sepetler)) * 100\n",
    "    print(f\"{i:2d}.  | {urun:<12} | {sayi:8d}     | {support:6.1f}%\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Popülarite grafiği\n",
    "plt.figure(figsize=(12, 8))\n",
    "\n",
    "# En popüler 10 ürün\n",
    "top_10 = sorted_urunler[:10]\n",
    "urunler = [item[0] for item in top_10]\n",
    "sayilar = [item[1] for item in top_10]\n",
    "\n",
    "bars = plt.bar(urunler, sayilar, color='skyblue', alpha=0.8)\n",
    "plt.title('En Popüler 10 Ürün', fontsize=16, fontweight='bold', pad=20)\n",
    "plt.xlabel('Ürünler', fontsize=14)\n",
    "plt.ylabel('Sepet Sayısı', fontsize=14)\n",
    "plt.xticks(rotation=45, ha='right')\n",
    "\n",
    "# Barların üzerine sayıları ekle\n",
    "for bar, sayi in zip(bars, sayilar):\n",
    "    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5, \n",
    "             str(sayi), ha='center', va='bottom', fontweight='bold')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.grid(axis='y', alpha=0.3)\n",
    "plt.show()\n",
    "\n",
    "print(f\"📊 En popüler ürün: {sorted_urunler[0][0]} ({sorted_urunler[0][1]} sepet)\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. 🔗 Support Kavramı\n",
    "\n",
    "**Support**, bir itemset'in (ürün veya ürün grubu) ne kadar sık görüldüğünü ölçer.\n",
    "\n",
    "### 📐 Formül:\n",
    "```\n",
    "Support(A) = A içeren sepet sayısı / Toplam sepet sayısı\n",
    "```\n",
    "\n",
    "### 💡 Yorumlama:\n",
    "- Support ne kadar yüksekse, o itemset o kadar popülerdir\n",
    "- Genellikle %5-10 ve üzeri support değerleri anlamlı kabul edilir"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Support hesaplama örneği\n",
    "def support_hesapla(urun, sepetler):\n",
    "    urun_sayisi = sum(1 for sepet in sepetler if urun in sepet)\n",
    "    support = urun_sayisi / len(sepetler)\n",
    "    return support\n",
    "\n",
    "# Örnek hesaplamalar\n",
    "print(\"🧮 SUPPORT HESAPLAMA ÖRNEKLERİ:\")\n",
    "print(\"-\" * 40)\n",
    "\n",
    "ornek_urunler = ['chocolate', 'Milk', 'Bread', 'Apple']\n",
    "\n",
    "for urun in ornek_urunler:\n",
    "    support = support_hesapla(urun, sepetler)\n",
    "    sepet_sayisi = sum(1 for sepet in sepetler if urun in sepet)\n",
    "    \n",
    "    print(f\"📦 {urun}:\")\n",
    "    print(f\"   Sepet sayısı: {sepet_sayisi}\")\n",
    "    print(f\"   Support: {support:.3f} (%{support*100:.1f})\")\n",
    "    print()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. 🤝 Ürün Birlikteliklerini Bulma\n",
    "\n",
    "Hangi ürünler birlikte satın alınıyor? İki ürünlü kombinasyonları inceleyelim."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def birliktelik_hesapla(sepetler, min_support=0.05):\n",
    "    \"\"\"İki ürünlü birliktelikleri hesaplar\"\"\"\n",
    "    toplam_sepet = len(sepetler)\n",
    "    min_sepet_sayisi = int(min_support * toplam_sepet)\n",
    "    \n",
    "    print(f\"🔍 Minimum support: %{min_support*100:.0f} ({min_sepet_sayisi} sepet)\")\n",
    "    \n",
    "    birliktelik_sayilari = {}\n",
    "    \n",
    "    # Her sepetteki ürün çiftlerini say\n",
    "    for sepet in sepetler:\n",
    "        if len(sepet) >= 2:\n",
    "            # Bu sepetteki tüm ürün çiftleri\n",
    "            for urun1, urun2 in combinations(sepet, 2):\n",
    "                # Alfabetik sıra (tutarlılık için)\n",
    "                if urun1 > urun2:\n",
    "                    urun1, urun2 = urun2, urun1\n",
    "                \n",
    "                cift = (urun1, urun2)\n",
    "                birliktelik_sayilari[cift] = birliktelik_sayilari.get(cift, 0) + 1\n",
    "    \n",
    "    # Minimum support'u geçenleri filtrele\n",
    "    onemli_birliktelikler = {}\n",
    "    for cift, sayi in birliktelik_sayilari.items():\n",
    "        if sayi >= min_sepet_sayisi:\n",
    "            support = sayi / toplam_sepet\n",
    "            onemli_birliktelikler[cift] = {\n",
    "                'sepet_sayisi': sayi,\n",
    "                'support': support\n",
    "            }\n",
    "    \n",
    "    return onemli_birliktelikler\n",
    "\n",
    "# Birliktelikleri hesapla\n",
    "birliktelikler = birliktelik_hesapla(sepetler, min_support=0.05)\n",
    "\n",
    "print(f\"\\n✅ {len(birliktelikler)} önemli birliktelik bulundu!\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# En güçlü birliktelikleri göster\n",
    "sorted_birliktelikler = sorted(birliktelikler.items(), \n",
    "                              key=lambda x: x[1]['support'], reverse=True)\n",
    "\n",
    "print(\"🤝 EN GÜÇLÜ BİRLİKTELİKLER:\")\n",
    "print(\"Sıra | Ürün Çifti                    | Sepet | Support\")\n",
    "print(\"-\" * 60)\n",
    "\n",
    "for i, (cift, bilgi) in enumerate(sorted_birliktelikler[:15], 1):\n",
    "    urun1, urun2 = cift\n",
    "    sepet_sayisi = bilgi['sepet_sayisi']\n",
    "    support = bilgi['support']\n",
    "    \n",
    "    print(f\"{i:2d}.  | {urun1:<12} + {urun2:<12} | {sepet_sayisi:3d}   | {support:5.1%}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Birliktelik grafiği\n",
    "plt.figure(figsize=(12, 8))\n",
    "\n",
    "# En güçlü 10 birliktelik\n",
    "top_10_birliktelik = sorted_birliktelikler[:10]\n",
    "\n",
    "cift_isimleri = [f\"{cift[0]} + {cift[1]}\" for cift, _ in top_10_birliktelik]\n",
    "sepet_sayilari = [bilgi['sepet_sayisi'] for _, bilgi in top_10_birliktelik]\n",
    "\n",
    "bars = plt.barh(cift_isimleri, sepet_sayilari, color='lightcoral', alpha=0.8)\n",
    "plt.title('En Güçlü 10 Ürün Birlikteliği', fontsize=16, fontweight='bold', pad=20)\n",
    "plt.xlabel('Sepet Sayısı', fontsize=14)\n",
    "plt.ylabel('Ürün Çiftleri', fontsize=14)\n",
    "\n",
    "# Sayıları barların yanına ekle\n",
    "for bar, sayi in zip(bars, sepet_sayilari):\n",
    "    plt.text(bar.get_width() + 1, bar.get_y() + bar.get_height()/2, \n",
    "             str(sayi), ha='left', va='center', fontweight='bold')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.grid(axis='x', alpha=0.3)\n",
    "plt.show()\n",
    "\n",
    "# En güçlü birliktelik\n",
    "en_guclu = sorted_birliktelikler[0]\n",
    "urun1, urun2 = en_guclu[0]\n",
    "print(f\"\\n🏆 En güçlü birliktelik: {urun1} + {urun2} (%{en_guclu[1]['support']*100:.1f})\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. 📋 Confidence Kavramı\n",
    "\n",
    "**Confidence**, \"A ürününü alan müşterilerin kaçta kaçının B ürününü de aldığını\" ölçer.\n",
    "\n",
    "### 📐 Formül:\n",
    "```\n",
    "Confidence(A → B) = Support(A,B) / Support(A)\n",
    "```\n",
    "\n",
    "### 💡 Yorumlama:\n",
    "- %50+ confidence genellikle anlamlı kabul edilir\n",
    "- \"A alan müşterilerin %X'i B'yi de alıyor\" şeklinde yorumlanır"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def kural_olustur(birliktelikler, urun_sayilari, toplam_sepet, min_confidence=0.3):\n",
    "    \"\"\"Association rules oluşturur\"\"\"\n",
    "    kurallar = []\n",
    "    \n",
    "    for (urun1, urun2), bilgi in birliktelikler.items():\n",
    "        birlikte_sayi = bilgi['sepet_sayisi']\n",
    "        \n",
    "        # Kural 1: urun1 → urun2\n",
    "        confidence1 = birlikte_sayi / urun_sayilari[urun1]\n",
    "        if confidence1 >= min_confidence:\n",
    "            lift1 = confidence1 / (urun_sayilari[urun2] / toplam_sepet)\n",
    "            kurallar.append({\n",
    "                'antecedent': urun1,      # Öncül (A)\n",
    "                'consequent': urun2,      # Sonuç (B) \n",
    "                'support': bilgi['support'],\n",
    "                'confidence': confidence1,\n",
    "                'lift': lift1\n",
    "            })\n",
    "        \n",
    "        # Kural 2: urun2 → urun1\n",
    "        confidence2 = birlikte_sayi / urun_sayilari[urun2]\n",
    "        if confidence2 >= min_confidence:\n",
    "            lift2 = confidence2 / (urun_sayilari[urun1] / toplam_sepet)\n",
    "            kurallar.append({\n",
    "                'antecedent': urun2,\n",
    "                'consequent': urun1,\n",
    "                'support': bilgi['support'],\n",
    "                'confidence': confidence2,\n",
    "                'lift': lift2\n",
    "            })\n",
    "    \n",
    "    return sorted(kurallar, key=lambda x: x['confidence'], reverse=True)\n",
    "\n",
    "# Kuralları oluştur\n",
    "kurallar = kural_olustur(birliktelikler, urun_sayilari, len(sepetler), min_confidence=0.3)\n",
    "\n",
    "print(f\"🔍 Minimum confidence: %30\")\n",
    "print(f\"✅ {len(kurallar)} association rule bulundu!\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# En güçlü kuralları göster\n",
    "print(\"📋 EN GÜÇLÜ ASSOCIATION RULES:\")\n",
    "print(\"Sıra | Öncül       → Sonuç       | Support | Confidence | Lift\")\n",
    "print(\"-\" * 70)\n",
    "\n",
    "for i, kural in enumerate(kurallar[:15], 1):\n",
    "    print(f\"{i:2d}.  | {kural['antecedent']:<12} → {kural['consequent']:<12} | \"\n",
    "          f\"{kural['support']:5.1%}   | {kural['confidence']:8.1%}   | {kural['lift']:4.2f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 💡 Confidence Açıklaması:\n",
    "\n",
    "Yukarıdaki ilk kural için örnek:\n",
    "- **Milk → chocolate**: %68 confidence\n",
    "- **Anlamı**: Milk alan müşterilerin %68'i chocolate da alıyor\n",
    "- **İş uygulaması**: Milk alan müşterilere chocolate önerebiliriz"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 8. 🚀 Lift Kavramı\n",
    "\n",
    "**Lift**, iki ürünün birlikte alınma olasılığının tesadüfi duruma göre ne kadar güçlü olduğunu ölçer.\n",
    "\n",
    "### 📐 Formül:\n",
    "```\n",
    "Lift(A → B) = Confidence(A → B) / Support(B)\n",
    "```\n",
    "\n",
    "### 💡 Yorumlama:\n",
    "- **Lift = 1**: Tesadüfi birliktelik (bağımsız)\n",
    "- **Lift > 1**: Pozitif birliktelik (birlikte alınma eğilimi var)\n",
    "- **Lift < 1**: Negatif birliktelik (birlikte alınma eğilimi yok)\n",
    "- **Lift > 1.5**: Güçlü pozitif birliktelik"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Lift değerlerini analiz et\n",
    "lift_degerleri = [kural['lift'] for kural in kurallar]\n",
    "\n",
    "print(\"🚀 LIFT ANALİZİ:\")\n",
    "print(f\"🔸 Ortalama Lift: {np.mean(lift_degerleri):.2f}\")\n",
    "print(f\"🔸 Maksimum Lift: {max(lift_degerleri):.2f}\")\n",
    "print(f\"🔸 Minimum Lift: {min(lift_degerleri):.2f}\")\n",
    "\n",
    "# Lift kategorileri\n",
    "guclu_kurallar = [k for k in kurallar if k['lift'] > 1.5]\n",
    "orta_kurallar = [k for k in kurallar if 1.0 < k['lift'] <= 1.5]\n",
    "zayif_kurallar = [k for k in kurallar if k['lift'] <= 1.0]\n",
    "\n",
    "print(f\"\\n📊 LIFT KATEGORİLERİ:\")\n",
    "print(f\"🟢 Güçlü kurallar (Lift > 1.5): {len(guclu_kurallar)}\")\n",
    "print(f\"🟡 Orta kurallar (1.0 < Lift ≤ 1.5): {len(orta_kurallar)}\")\n",
    "print(f\"🔴 Zayıf kurallar (Lift ≤ 1.0): {len(zayif_kurallar)}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Lift histogramı\n",
    "plt.figure(figsize=(10, 6))\n",
    "\n",
    "plt.hist(lift_degerleri, bins=20, alpha=0.7, color='lightgreen', edgecolor='black')\n",
    "plt.axvline(x=1.0, color='red', linestyle='--', linewidth=2, label='Lift = 1 (Tesadüfi)')\n",
    "plt.axvline(x=1.5, color='orange', linestyle='--', linewidth=2, label='Lift = 1.5 (Güçlü)')\n",
    "\n",
    "plt.title('Lift Değerlerinin Dağılımı', fontsize=16, fontweight='bold')\n",
    "plt.xlabel('Lift Değeri', fontsize=12)\n",
    "plt.ylabel('Kural Sayısı', fontsize=12)\n",
    "plt.legend()\n",
    "plt.grid(alpha=0.3)\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "print(f\"📊 Lift grafiği: Çoğu kural 1.0'dan büyük lift değerine sahip (pozitif birliktelik)\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# En güçlü kuralların detaylı açıklaması\n",
    "print(\"🏆 EN GÜÇLÜ 3 KURALIN DETAYLI AÇIKLAMASI:\")\n",
    "print(\"=\" * 60)\n",
    "\n",
    "for i, kural in enumerate(kurallar[:3], 1):\n",
    "    oncul = kural['antecedent']\n",
    "    sonuc = kural['consequent']\n",
    "    confidence = kural['confidence']\n",
    "    lift = kural['lift']\n",
    "    support = kural['support']\n",
    "    \n",
    "    print(f\"\\n{i}. KURAL: {oncul} → {sonuc}\")\n",
    "    print(f\"   📊 Support: %{support*100:.1f} (Sepetlerin %{support*100:.1f}'inde birlikte görülüyor)\")\n",
    "    print(f\"   🎯 Confidence: %{confidence*100:.1f} ({oncul} alan müşterilerin %{confidence*100:.0f}'i {sonuc} da alıyor)\")\n",
    "    print(f\"   🚀 Lift: {lift:.2f} (Tesadüften {lift:.1f} kat daha güçlü birliktelik)\")\n",
    "    \n",
    "    print(f\"   💡 İş Uygulaması:\")\n",
    "    print(f\"      - {oncul} alan müşterilere {sonuc} önerebilirsiniz\")\n",
    "    print(f\"      - Bu ürünleri mağazada yakın yerlere koyabilirsiniz\")\n",
    "    print(f\"      - {oncul} + {sonuc} için birlikte kampanya yapabilirsiniz\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 9. 🎯 Ürün Önerisi Sistemi\n",
    "\n",
    "Artık bir ürün için öneriler üretebiliriz!"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def urun_onerisi_uret(secilen_urun, kurallar, top_n=5):\n",
    "    \"\"\"Belirli bir ürün için öneriler üretir\"\"\"\n",
    "    \n",
    "    # Bu ürün için geçerli kuralları bul\n",
    "    uygun_kurallar = [kural for kural in kurallar \n",
    "                     if kural['antecedent'] == secilen_urun]\n",
    "    \n",
    "    if not uygun_kurallar:\n",
    "        return []\n",
    "    \n",
    "    # Confidence'a göre sırala\n",
    "    uygun_kurallar.sort(key=lambda x: x['confidence'], reverse=True)\n",
    "    \n",
    "    return uygun_kurallar[:top_n]\n",
    "\n",
    "# Örnek öneriler\n",
    "ornek_urunler = ['Milk', 'Bread', 'chocolate', 'Apple']\n",
    "\n",
    "for urun in ornek_urunler:\n",
    "    print(f\"\\n🎯 '{urun}' İÇİN ÖNERİLER:\")\n",
    "    print(\"-\" * 40)\n",
    "    \n",
    "    oneriler = urun_onerisi_uret(urun, kurallar, top_n=3)\n",
    "    \n",
    "    if oneriler:\n",
    "        for i, kural in enumerate(oneriler, 1):\n",
    "            print(f\"{i}. {kural['consequent']}\")\n",
    "            print(f\"   Güven: %{kural['confidence']*100:.1f}\")\n",
    "            print(f\"   Lift: {kural['lift']:.2f}\")\n",
    "            print(f\"   Açıklama: '{urun}' alan müşterilerin %{kural['confidence']*100:.0f}'i '{kural['consequent']}' da alıyor\")\n",
    "            print()\n",
    "    else:\n",
    "        print(f\"   ❌ '{urun}' için öneri bulunamadı\")\n",
    "        print(f\"   Bu ürün başka ürünlerle güçlü birliktelik oluşturmuyor\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# İnteraktif öneri sistemi\n",
    "def onerileri_gorsellestir(secilen_urun, kurallar):\n",
    "    \"\"\"Önerileri grafik olarak gösterir\"\"\"\n",
    "    \n",
    "    oneriler = urun_onerisi_uret(secilen_urun, kurallar, top_n=5)\n",
    "    \n",
    "    if not oneriler:\n",
    "        print(f\"❌ '{secilen_urun}' için öneri bulunamadı\")\n",
    "        return\n",
    "    \n",
    "    # Grafik verilerini hazırla\n",
    "    urun_isimleri = [kural['consequent'] for kural in oneriler]\n",
    "    confidence_skorlari = [kural['confidence']*100 for kural in oneriler]\n",
    "    \n",
    "    # Grafik çiz\n",
    "    plt.figure(figsize=(10, 6))\n",
    "    bars = plt.barh(urun_isimleri, confidence_skorlari, color='lightblue', alpha=0.8)\n",
    "    \n",
    "    plt.title(f\"'{secilen_urun}' için Ürün Önerileri\", fontsize=16, fontweight='bold')\n",
    "    plt.xlabel('Confidence Skoru (%)', fontsize=12)\n",
    "    plt.ylabel('Önerilen Ürünler', fontsize=12)\n",
    "    \n",
    "    # Skorları barların yanına ekle\n",
    "    for bar, skor in zip(bars, confidence_skorlari):\n",
    "        plt.text(bar.get_width() + 1, bar.get_y() + bar.get_height()/2, \n",
    "                 f'{skor:.1f}%', ha='left', va='center', fontweight='bold')\n",
    "    \n",
    "    plt.tight_layout()\n",
    "    plt.grid(axis='x', alpha=0.3)\n",
    "    plt.show()\n",
    "    \n",
    "    return oneriler\n",
    "\n",
    "# Milk için önerileri görselleştir\n",
    "print(\"📊 Milk için öneriler:\")\n",
    "milk_onerileri = onerileri_gorsellestir('Milk', kurallar)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 10. 📊 Sonuçların Özeti ve İş Uygulamaları"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Genel özet raporu\n",
    "print(\"📋 MARKET BASKET ANALİZİ ÖZET RAPORU\")\n",
    "print(\"=\" * 50)\n",
    "\n",
    "print(f\"📊 VERİ ÖZETİ:\")\n",
    "print(f\"   🔸 Toplam sepet sayısı: {len(sepetler):,}\")\n",
    "print(f\"   🔸 Toplam ürün çeşidi: {len(urun_sayilari)}\")\n",
    "print(f\"   🔸 Ortalama ürün/sepet: {np.mean(sepet_boyutlari):.1f}\")\n",
    "\n",
    "print(f\"\\n🤝 BİRLİKTELİK ANALİZİ:\")\n",
    "print(f\"   🔸 Bulunan ürün çifti birlikteliği: {len(birliktelikler)}\")\n",
    "print(f\"   🔸 Minimum support eşiği: %5\")\n",
    "\n",
    "print(f\"\\n📋 ASSOCIATION RULES:\")\n",
    "print(f\"   🔸 Toplam kural sayısı: {len(kurallar)}\")\n",
    "print(f\"   🔸 Minimum confidence eşiği: %30\")\n",
    "print(f\"   🔸 Güçlü kurallar (Lift > 1.5): {len(guclu_kurallar)}\")\n",
    "\n",
    "# En popüler ürün\n",
    "en_populer_urun, en_populer_sayi = sorted_urunler[0]\n",
    "print(f\"\\n🏆 EN POPÜLER ÜRÜN:\")\n",
    "print(f\"   {en_populer_urun}: {en_populer_sayi} sepet (%{(en_populer_sayi/len(sepetler))*100:.1f})\")\n",
    "\n",
    "# En güçlü birliktelik\n",
    "en_guclu_cift, en_guclu_bilgi = sorted_birliktelikler[0]\n",
    "print(f\"\\n🤝 EN GÜÇLÜ BİRLİKTELİK:\")\n",
    "print(f\"   {en_guclu_cift[0]} + {en_guclu_cift[1]}: %{en_guclu_bilgi['support']*100:.1f} support\")\n",
    "\n",
    "# En güçlü kural\n",
    "en_guclu_kural = kurallar[0]\n",
    "print(f\"\\n📋 EN GÜÇLÜ KURAL:\")\n",
    "print(f\"   {en_guclu_kural['antecedent']} → {en_guclu_kural['consequent']}: %{en_guclu_kural['confidence']*100:.1f} confidence\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# İş önerileri\n",
    "print(\"\\n💼 İŞ ÖNERİLERİ VE UYGULAMALAR:\")\n",
    "print(\"=\" * 50)\n",
    "\n",
    "print(\"🛒 CROSS-SELLING (ÇAPRAZ SATIŞ):\")\n",
    "for i, kural in enumerate(kurallar[:3], 1):\n",
    "    print(f\"   {i}. {kural['antecedent']} alan müşterilere {kural['consequent']} önerin\")\n",
    "    print(f\"      (%{kural['confidence']*100:.0f} başarı oranı beklenir)\")\n",
    "\n",
    "print(f\"\\n🏪 MAĞAZA DÜZENİ:\")\n",
    "for i, (cift, bilgi) in enumerate(sorted_birliktelikler[:3], 1):\n",
    "    urun1, urun2 = cift\n",
    "    print(f\"   {i}. {urun1} ve {urun2}'ı yakın raflara koyun\")\n",
    "    print(f\"      (Sepetlerin %{bilgi['support']*100:.1f}'inde birlikte alınıyor)\")\n",
    "\n",
    "print(f\"\\n🎯 KAMPANYA PLANLAMA:\")\n",
    "print(f\"   1. En popüler ürün {en_populer_urun} için özel kampanya\")\n",
    "print(f\"   2. {en_guclu_cift[0]} + {en_guclu_cift[1]} kombo paketi oluşturun\")\n",
    "print(f\"   3. Güçlü kurallar için '2. ürün %50 indirim' kampanyası\")\n",
    "\n",
    "print(f\"\\n📊 STOK YÖNETİMİ:\")\n",
    "print(f\"   1. Birlikte alınan ürünlerin stoklarını birlikte planlayın\")\n",
    "print(f\"   2. {en_populer_urun} bitiyorsa, birlikte alınan ürünler de bitebilir\")\n",
    "print(f\"   3. Mevsimsel değişiklikler için bu analizleri tekrarlayın\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 11. 🎓 Öğrenim Değerlendirmesi\n",
    "\n",
    "### ✅ Bu Tutorial'da Öğrendikleriniz:\n",
    "\n",
    "1. **Market Basket Analysis'in temel kavramları**\n",
    "   - Support: Popülarite ölçüsü\n",
    "   - Confidence: Koşullu olasılık\n",
    "   - Lift: Birliktelik gücü\n",
    "\n",
    "2. **Veri hazırlama ve analiz süreci**\n",
    "   - Binary matrix formatından sepet listelerine dönüşüm\n",
    "   - Ürün popülaritesi hesaplama\n",
    "   - Birliktelik matrisi oluşturma\n",
    "\n",
    "3. **Association Rules oluşturma**\n",
    "   - İki yönlü kurallar (A→B ve B→A)\n",
    "   - Threshold belirleme (min_support, min_confidence)\n",
    "   - Kural kalitesi değerlendirme\n",
    "\n",
    "4. **Sonuçları yorumlama ve iş uygulamaları**\n",
    "   - Cross-selling stratejileri\n",
    "   - Mağaza düzeni optimizasyonu\n",
    "   - Kampanya planlama\n",
    "   - Stok yönetimi\n",
    "\n",
    "### 🚀 Sonraki Adımlar:\n",
    "- Daha büyük dataset'lerle çalışma\n",
    "- 3+ ürünlü kombinasyonları analiz etme\n",
    "- Zaman bazlı analiz (hangi ürünler hangi dönemlerde birlikte alınıyor?)\n",
    "- Müşteri segmentasyonu ile birleştirme\n",
    "- Gerçek e-ticaret verisi ile uygulama"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Tutorial tamamlandı mesajı\n",
    "print(\"🎉 TUTORIAL TAMAMLANDI! 🎉\")\n",
    "print(\"=\"*40)\n",
    "print(\"✅ Market Basket Analysis temellerini öğrendiniz\")\n",
    "print(\"✅ Support, Confidence, Lift kavramlarını anladınız\")\n",
    "print(\"✅ Association Rules oluşturmayı öğrendiniz\")\n",
    "print(\"✅ Sonuçları iş uygulamalarına dönüştürmeyi öğrendiniz\")\n",
    "print(\"\")\n",
    "print(\"🚀 Artık kendi verilerinizle analiz yapabilirsiniz!\")\n",
    "print(\"📚 Daha ileri konular için ek kaynakları inceleyin\")\n",
    "print(\"💡 Sorularınız için eğitmeninizle iletişime geçin\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}